In [1]:
# this is my first shoot in loanPred III competition thanks to AV for this opportnuty 
import pandas as pd
import seaborn as sbs
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

C:\Users\h.mortazavi\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### remove NAN values in train set

In [4]:
for col in train.columns:
    
    print(col,sum(train[col].isnull()))
    

Loan_ID 0
Gender 13
Married 3
Dependents 15
Education 0
Self_Employed 32
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 22
Loan_Amount_Term 14
Credit_History 50
Property_Area 0
Loan_Status 0


In [5]:
# replace NA LoanAmount with mean
train.loc[train.LoanAmount.isnull(),'LoanAmount']=np.mean(train.loc[train.LoanAmount.notnull(),
                                                                    'LoanAmount'] ) 

In [6]:
train[train.Dependents.isnull()]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
102,LP001350,Male,Yes,NaN,Graduate,No,13650,0.0,146.412162,360.0,1.0,Urban,Y
104,LP001357,Male,NaN,NaN,Graduate,No,3816,754.0,160.000000,360.0,1.0,Urban,Y
120,LP001426,Male,Yes,NaN,Graduate,No,5667,2667.0,180.000000,360.0,1.0,Rural,Y
226,LP001754,Male,Yes,NaN,Not Graduate,Yes,4735,0.0,138.000000,360.0,1.0,Urban,N
228,LP001760,Male,NaN,NaN,Graduate,No,4758,0.0,158.000000,480.0,1.0,Semiurban,Y
293,LP001945,Female,No,NaN,Graduate,No,5417,0.0,143.000000,480.0,0.0,Urban,N
301,LP001972,Male,Yes,NaN,Not Graduate,No,2875,1750.0,105.000000,360.0,1.0,Semiurban,Y
332,LP002100,Male,No,NaN,Graduate,No,2833,0.0,71.000000,360.0,1.0,Urban,Y
335,LP002106,Male,Yes,NaN,Graduate,Yes,5503,4490.0,70.000000,NaN,1.0,Semiurban,Y
346,LP002130,Male,Yes,NaN,Not Graduate,No,3523,3230.0,152.000000,360.0,0.0,Rural,N


In [7]:
# not married and Female have not independent 
train.loc[(train.Dependents.isnull()) & (  train.Married=='No'),'Dependents']=str(0)
train.loc[(train.Dependents.isnull()) & (  train.Gender=='Female'),'Dependents']=str(0)
# dependent nan replace with 0
train.loc[(train.Dependents.isnull())]=str(0)

In [8]:

train.loc[train.Gender.isnull(),'Gender']='Male'
train.loc[train.Married.isnull(),'Married']='Yes'
train.loc[train.Credit_History.isnull(),'Credit_History']=1.0
train.loc[train.Loan_Amount_Term.isnull(),'Loan_Amount_Term']=360
train.loc[train.Self_Employed.isnull(),'Self_Employed']='No'


### remove NAN values in train set

In [9]:
for col in train.columns:
    
    print(col,sum(train[col].isnull()))

Loan_ID 0
Gender 0
Married 0
Dependents 0
Education 0
Self_Employed 0
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 0
Loan_Amount_Term 0
Credit_History 0
Property_Area 0
Loan_Status 0


### transform train data

In [10]:
def transform(data):
    data.Loan_Status=data.Loan_Status.map({'N':0,'Y':1})
    data.Gender=data.Gender.map({'Male':1,'Female':2})
    data.Married=data.Married.map({'Yes':1,'No':0})
    data.Education=data.Education.map({'Graduate':1,'Not Graduate':0})
    data.Self_Employed=data.Self_Employed.map({'No':0,'yes':1})
    data.Dependents=data.Dependents.map({'3+':3,'1':1,'2':2,'0':0})
    data.Property_Area=data.Property_Area.map({'Urban':1,'Semiurban':2,'Rural':3})
    return data
train=transform(train)
train=train.dropna()
train['income']=train['ApplicantIncome']+train['CoapplicantIncome']

y=train.Loan_Status
x=train.drop(['Loan_ID','Loan_Status','ApplicantIncome','CoapplicantIncome'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=5)
logreg1=LogisticRegression(penalty='l1',C=.1)
cross_val_score(logreg1,x,y,scoring='accuracy').mean()

0.80157175200278641

In [11]:
def normalize(data):
    return data/max(data)


In [12]:
x.LoanAmount=normalize(x.LoanAmount)


In [13]:
x.Loan_Amount_Term=normalize(x.Loan_Amount_Term)
x.income=normalize(np.int64(x.income))

In [14]:
clf=logreg1.fit(x,y)

In [15]:
x.head()

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,income
0,1.0,0.0,0,1.0,0.0,0.20916,0.75,1,1.0,0.072210
1,1.0,1.0,1,1.0,0.0,0.182857,0.75,1,3.0,0.075198
3,1.0,1.0,0,0.0,0.0,0.171429,0.75,1,1.0,0.061000
4,1.0,0.0,0,1.0,0.0,0.201429,0.75,1,1.0,0.074074
6,1.0,1.0,0,0.0,0.0,0.135714,0.75,1,1.0,0.047519


#### prediction on test data

In [16]:

def transform_test(data):
    data.Gender=data.Gender.map({'Male':1,'Female':2})
    data.Married=data.Married.map({'Yes':1,'No':0})
    data.Education=data.Education.map({'Graduate':1,'Not Graduate':0})
    data.Self_Employed=data.Self_Employed.map({'No':0,'yes':1})
    data.Dependents=data.Dependents.map({'3+':3,'1':1,'2':2,'0':0})
    data.Property_Area=data.Property_Area.map({'Urban':1,'Semiurban':2,'Rural':3})
    return data

test=transform_test(test)

In [17]:
# replace NA LoanAmount with mean
test.loc[test.LoanAmount.isnull(),'LoanAmount']=np.mean(np.float64(test.loc[test.LoanAmount.notnull(),
                                                                    'LoanAmount']) ) 

In [18]:
test.LoanAmount=test.LoanAmount.astype(np.float64)


In [19]:
test[test.Dependents.isnull()]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
46,LP001237,1.0,1,NaN,0,0.0,4163,1475,162.000000,360.0,1.0,1
70,LP001366,2.0,0,NaN,1,0.0,3250,0,95.000000,360.0,1.0,2
111,LP001587,1.0,1,NaN,1,0.0,4082,0,93.000000,360.0,1.0,2
138,LP001769,NaN,0,NaN,1,0.0,3333,1250,110.000000,360.0,1.0,2
202,LP002111,1.0,1,NaN,1,0.0,3016,1300,100.000000,360.0,NaN,1
247,LP002360,1.0,1,NaN,1,0.0,10000,0,136.132597,360.0,1.0,1
251,LP002385,1.0,1,NaN,1,0.0,3863,0,70.000000,300.0,1.0,2
265,LP002441,1.0,0,NaN,1,0.0,3579,3308,138.000000,360.0,NaN,2
302,LP002654,2.0,0,NaN,1,NaN,14987,0,177.000000,360.0,1.0,3
312,LP002754,1.0,0,NaN,1,0.0,2066,2108,104.000000,84.0,1.0,1


In [51]:
test.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome      int64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
dtype: object

In [20]:
# not married and Female have not independent 
test.loc[(test.Dependents.isnull()) & (  test.Married==0),'Dependents']=0
test.loc[(test.Dependents.isnull()) & (  test.Gender==0),'Dependents']=0
# dependent nan replace with 0
test.loc[(test.Dependents.isnull())]=0

In [21]:

test.loc[test.Gender.isnull(),'Gender']=1
test.loc[test.Married.isnull(),'Married']=1
test.loc[test.Credit_History.isnull(),'Credit_History']=1
test.loc[test.Loan_Amount_Term.isnull(),'Loan_Amount_Term']=360
test.loc[test.Self_Employed.isnull(),'Self_Employed']=0

In [22]:
for col in x_test.columns:
    
    print(col,sum(x_test[col].isnull()))

Gender 0
Married 0
Dependents 0
Education 0
Self_Employed 0
LoanAmount 0
Loan_Amount_Term 0
Credit_History 0
Property_Area 0
income 0


In [23]:
test['income']=test['ApplicantIncome']+test['CoapplicantIncome']
x_test=test.drop(['Loan_ID','ApplicantIncome','CoapplicantIncome'],axis=1)


In [25]:
x_test.LoanAmount=normalize(x_test.LoanAmount)
x_test.Loan_Amount_Term=normalize(x_test.Loan_Amount_Term)
x_test.income=normalize(x_test.income)

In [27]:
pred=clf.predict(x_test)

In [36]:
test_in=pd.read_csv('test.csv')
result=pd.DataFrame({'Loan_ID':test_in.Loan_ID,'Loan_Status':pred})

In [37]:
result.Loan_Status=result.Loan_Status.map({1:'Y',0:'N'})
result.to_csv('ressult.csv',sep=',',index=False)

In [46]:
# prediction with sklearn(RF)
from sklearn.ensemble import RandomForestClassifier


In [47]:
clf_rf=RandomForestClassifier()

clf_rf.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [63]:
rf_pred=clf_rf.predict(x_test)

In [64]:
x_test.head()

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,income
0,1.0,1,0.0,1,0.0,0.200000,0.75,1.0,1,0.078865
1,1.0,1,1.0,1,0.0,0.229091,0.75,1.0,1,0.063092
2,1.0,1,2.0,1,0.0,0.378182,0.75,1.0,1,0.093756
3,1.0,1,2.0,1,0.0,0.181818,0.75,1.0,1,0.067366
4,1.0,0,0.0,0,0.0,0.141818,0.75,1.0,1,0.045168


In [65]:
result_rf=pd.DataFrame({'Loan_ID':test_in.Loan_ID,'Loan_Status':rf_pred})

In [66]:
result_rf.Loan_Status=result_rf.Loan_Status.map({1:'Y',0:'N'})
result_rf.to_csv('result_rf.csv',sep=',',index=False)
